In [5]:
import os
import pandas as pd

In [6]:
from src.tabulka import tabulka

In [7]:
df = pd.read_json(os.path.join("data","cro_raw.json"))

In [8]:
df.shape

(1087914, 7)

In [9]:
df['since'] = pd.to_datetime(df['since'].apply(lambda x: x.replace("02:00","01:00")))

In [10]:
df['interpret'] = df['interpret'].apply(lambda x: x.upper())
df['track'] = df['track'].apply(lambda x: x.upper())

In [11]:
df.head(2)

,since,id,interpret,interpret_id,track,track_id,stanice
0,2020-12-01 04:42:44+01:00,10236811,"SAMUEL RAMEY & LONDON PHILHARMONIC ORCHESTRA, ...",24978,"NON PIU ANDRAI, ÁRIE Z 1. DĚJSTVÍ OPERY FIGARO",83837,d
1,2020-12-08 18:25:47+01:00,10270691,"SAMUEL RAMEY & LONDON PHILHARMONIC ORCHESTRA, ...",24978,"NON PIU ANDRAI, ÁRIE Z 1. DĚJSTVÍ OPERY FIGARO",83837,d


In [12]:
df['rok'] = df['since'].dt.year

In [13]:
df.groupby(["interpret","track"]).size().nlargest(20)

interpret                      track                
STUDIOVÝ ORCHESTR              ČESKÁ STÁTNÍ HYMNA       1000
NEDIVOČ                        FOTKY Z VÝLETU            660
ANETA LANGEROVÁ                BÍLÝ DEN                  652
HARRY STYLES                   AS IT WAS                 637
DAVID STYPKA                   FARMÁŘŮM                  611
MIRAI                          I PŘES TO VŠECHNO         600
MILEY CYRUS                    ANGELS LIKE YOU           591
JC STEWART                     I NEED YOU TO HATE ME     568
COLDPLAY                       HIGHER POWER              564
MILOW                          WHATEVER IT TAKES         535
CHINASKI                       BÁSEŇ                     529
DUA LIPA                       LOVE AGAIN                527
ADELE                          EASY ON ME                521
JELEN                          JEŠTĚ JEDNU NOC           517
ZAZ                            IMAGINE                   513
IMAGINE DRAGONS                F

In [14]:
cro22 = df[df['rok'] == 2022].groupby("interpret").size().nlargest(60)
cro22 = cro22[cro22 >= 1000]
cro22 = cro22.reset_index()
cro22['interpret'] = cro22['interpret'].apply(lambda x: x.title())
", ".join(cro22['interpret'].to_list())

'Karel Gott, Hana Zagorová, Waldemar Matuška, Chinaski, Marie Rottrová, Adele, Václav Neckář, Helena Vondráčková, Michal Tučný, Robbie Williams, Karel Zich'

In [15]:
df.groupby("interpret").size().nlargest(60)

interpret
CHINASKI              3502
KAREL GOTT            3190
ROBBIE WILLIAMS       2927
ADELE                 2891
HANA ZAGOROVÁ         2859
WALDEMAR MATUŠKA      2756
QUEEN                 2705
ANETA LANGEROVÁ       2702
VÁCLAV NECKÁŘ         2684
MARIE ROTTROVÁ        2601
HELENA VONDRÁČKOVÁ    2554
ED SHEERAN            2534
MIRAI                 2534
ROXETTE               2491
R.E.M.                2448
JELEN                 2364
BRYAN ADAMS           2318
COLDPLAY              2259
MARTA KUBIŠOVÁ        2254
MICHAL HRŮZA          2229
MICHAL TUČNÝ          2219
OLYMPIC               2217
MAREK ZTRACENÝ        2179
KAREL ZICH            2137
U 2                   2073
LUCIE                 2044
PETR REZEK            2039
P!NK                  2032
XINDL X               2030
PETR SPÁLENÝ          1994
VĚRA ŠPINAROVÁ        1977
JIŘÍ KORN             1974
ABBA                  1971
GEORGE EZRA           1954
CRANBERRIES           1928
PHIL COLLINS          1889
IMAGINE DRAGONS   

In [16]:
df.groupby(["interpret","track"]).size().nlargest(50)

interpret                      track                          
STUDIOVÝ ORCHESTR              ČESKÁ STÁTNÍ HYMNA                 1000
NEDIVOČ                        FOTKY Z VÝLETU                      660
ANETA LANGEROVÁ                BÍLÝ DEN                            652
HARRY STYLES                   AS IT WAS                           637
DAVID STYPKA                   FARMÁŘŮM                            611
MIRAI                          I PŘES TO VŠECHNO                   600
MILEY CYRUS                    ANGELS LIKE YOU                     591
JC STEWART                     I NEED YOU TO HATE ME               568
COLDPLAY                       HIGHER POWER                        564
MILOW                          WHATEVER IT TAKES                   535
CHINASKI                       BÁSEŇ                               529
DUA LIPA                       LOVE AGAIN                          527
ADELE                          EASY ON ME                          521
JELEN         

## Kánon hlavního proudu

In [17]:
univerzalove = df.groupby(['interpret',"track"])['stanice'].nunique()
univerzalove = univerzalove[univerzalove >= 3]

In [18]:
len(univerzalove)

229

In [19]:
univerzalove[univerzalove > 3]

interpret                track                    
EDITH PIAF               NON, JE NE REGRETTE RIEN     4
GEORGE HARRISON          MY SWEET LORD                4
GOLDEN KIDS              ČASY SE MĚNÍ                 4
HANA HEGEROVÁ            POTMĚŠILÝ HOST               4
JIŘÍ PAVLICA, HRADIŠŤAN  MODLITBA ZA VODU             4
JIŘÍ SCHELINGER          HOLUBÍ DŮM                   4
KAREL KRYL               BRATŘÍČKU, ZAVÍREJ VRÁTKA    4
LEONARD COHEN            SUZANNE                      4
MIROSLAV ŽBIRKA          BIELY KVET                   4
YVONNE PŘENOSILOVÁ       TAK PRÁZDNÁ                  4
Name: stanice, dtype: int64

In [20]:
univerzalove.head(25)

interpret          track                 
ABBA               CHIQUITITA                3
                   DANCING QUEEN             3
                   FERNANDO                  3
                   I HAVE A DREAM            3
                   SUPER TROUPER             3
ADAM PLACHETKA     ANDĚL PÁNĚ ZPRÁVU NESE    3
ADELE              EASY ON ME                3
                   HELLO                     3
                   SKYFALL                   3
ALANIS MORISSETTE  IRONIC                    3
ALPHAVILLE         FOREVER YOUNG             3
ANASTACIA          LEFT OUTSIDE ALONE        3
ANETA LANGEROVÁ    PODZIM                    3
                   VODA ŽIVÁ                 3
ARCHIES            SUGAR, SUGAR              3
AVRIL LAVIGNE      COMPLICATED               3
BEACH BOYS         GOD ONLY KNOWS            3
                   GOOD VIBRATIONS           3
BEATLES            ALL YOU NEED IS LOVE      3
                   CAN'T BUY ME LOVE         3
                  

## Mám rád

In [21]:
def hraje(x):
    return df[df['interpret'].str.lower().str.contains(x.lower(), na=False)].groupby(['interpret','track','stanice']).size().nlargest(1000).head(50)

In [22]:
hraje('bowie')

interpret                  track                                         stanice    
QUEEN, DAVID BOWIE         UNDER PRESSURE                                radiozurnal    123
DAVID BOWIE                CHINA GIRL                                    dvojka          12
                           LET'S DANCE                                   dvojka          11
                           LAZARUS                                       vltava           9
                           I CAN'T GIVE EVERYTHING AWAY                  vltava           8
                           HEROES                                        radiowave        8
QUEEN/DAVID BOWIE          UNDER PRESSURE                                dvojka           7
DAVID BOWIE                THE STARS (ARE OUT TONIGHT)                   vltava           7
                           GIRL LOVES ME                                 vltava           7
                           DOLLAR DAYS                                   vltava        

In [23]:
hraje('mišík')

interpret                                              track                                                                        stanice    
VLADIMÍR MIŠÍK                                         JEDNOU                                                                       radiozurnal    266
VLADIMÍR MIŠÍK, BLUE EFFECT                            SLUNEČNÝ HROB                                                                radiozurnal     88
VLADIMÍR MIŠÍK/ETC...                                  VARIACE NA RENESANČNÍ TÉMA                                                   dvojka          80
VLADIMÍR MIŠÍK                                         VARIACE NA RENESANČNÍ TÉMA                                                   radiozurnal     75
                                                       JEDNOU                                                                       dvojka          74
VLADIMÍR MIŠÍK, ETC...                                 STŘÍHALI DOHOLA MALÉHO CHLAPEČKA              

In [24]:
hraje('rolling')

interpret                        track                                           stanice    
ROLLING STONES                   ANYBODY SEEN MY BABY                            radiozurnal    219
                                 START ME UP                                     radiozurnal    207
                                 ANGIE                                           radiozurnal    165
                                                                                 dvojka          68
SLOWLY ROLLING CAMERA            PROTAGONIST                                     jazz            56
ROLLING STONES                   ANGRY                                           radiozurnal     33
                                 ANYBODY SEEN MY BABY                            dvojka          32
ROLLING BLACKOUTS COASTAL FEVER  CAMEO                                           radiowave       25
ROLLING STONES                   LIKE A ROLLING STONE                            dvojka          22
       

In [25]:
hraje('bob dylan')

interpret  track                                               stanice    
BOB DYLAN  SUBTERRANEAN HOMESICK BLUES                         pohoda         23
           I WANT YOU                                          pohoda         20
           LAY LADY LAY                                        pohoda         18
           BLOWIND IN THE WIND                                 pohoda         17
           LIKE A ROLLING STONE                                pohoda         16
           FOREVER YOUNG                                       vltava          9
           KNOCKIN ON HEAVEN S DOOR                            dvojka          8
           I WANT YOU                                          dvojka          8
           IT AIN'T ME BABE                                    vltava          7
           IT TAKES A LOT TO LAUGH, IT TAKES A TRAIN TO CRY    vltava          7
           MOTHER OF MUSES                                     vltava          7
           FALSE PROPHET          

In [26]:
hraje('kubiš')

interpret                                          track                              stanice    
MARTA KUBIŠOVÁ                                     DEPEŠE                             pohoda         161
                                                   NEPIŠ DÁL                          pohoda         152
                                                   POJĎ A MĚJ MNĚ RÁD                 pohoda         140
                                                   LOUDÁ SE PŮLMĚSÍC                  pohoda         113
                                                   NENÍ TO LÁSKA                      pohoda         111
                                                   RING-O-DING                        pohoda         108
                                                   NECHTE ZVONY ZNÍT                  pohoda          99
                                                   CESTA                              pohoda          96
                                                   NECHTE ZVON

In [27]:
hraje('velvet')

interpret                  track                              stanice    
VELVET UNDERGROUND & NICO  SUNDAY MORNING                     pohoda         22
THE VELVET UNDERGROUND     ALL TOMORROW'S PARTIES (LOU REED)  vltava          9
                           JESUS (LOU REED & DOUG YULE)       vltava          6
RED VELVET                 QUEENDOM                           radiowave       1
THE VELVET UNDERGROUND     SWEET JANE                         radiowave       1
VELVET HONEY               IT'S TIME TO BREATHE               radiozurnal     1
VELVET UNDERGROUND & NICO  I'LL BE YOUR MIRROR                dvojka          1
                                                              vltava          1
                           SUNDAY MORNING                     radiowave       1
dtype: int64

In [28]:
hraje('orchestr')

interpret                                                       track                                                                                stanice  
STUDIOVÝ ORCHESTR                                               ČESKÁ STÁTNÍ HYMNA                                                                   dvojka       1000
COTATCHA ORCHESTRA                                              SEN SEI                                                                              jazz          218
JAZZ AT LINCOLN CENTER ORCHESTRA & WYNTON MARSALIS              THE THREE MARIAS                                                                     jazz          200
COTATCHA ORCHESTRA                                              BILLY'S PILGRIMAGE                                                                   jazz          190
ORCHESTRE NATIONAL DE JAZZ                                      WONDER TWIN POWERS ACTIVATE!                                                         jazz          179
MATTHE

In [29]:
hraje('kryl')

interpret   track                                           stanice    
KAREL KRYL  BRATŘÍČKU, ZAVÍREJ VRÁTKA                       dvojka         8
            MARTINA                                         dvojka         8
            MORITURI TE SALUTANT                            dvojka         6
            ANDĚL                                           dvojka         3
            BRATŘÍČKU, ZAVÍREJ VRÁTKA                       radiozurnal    3
            PÍSEŇ O ŽRÁDLE                                  vltava         3
            BRATŘÍČKU,  ZAVÍREJ VRÁTKA                      dvojka         2
            SAMETOVÉ JARO                                   vltava         2
            KARAVANA MRAKŮ (ADMIN)                          vltava         2
            KARAVANA MRAKŮ                                  dvojka         2
            BRATŘÍČKU, ZAVÍREJ VRÁTKA                       vltava         2
            KARAVANA MRAKŮ                                  radiozurnal    1
    

In [30]:
hraje('nohavi')

interpret                         track                               stanice    
JAROMÍR NOHAVICA                  MÁMA MI NA KRK DALA KLÍČ            dvojka         109
                                  KOMETA                              radiozurnal     98
                                  MÁM JIZVU NA RTU                    radiozurnal     97
JAROMÍR NOHAVICA,  ČECHOMOR       BÁSNÍŘKA                            radiozurnal     84
JAROMÍR NOHAVICA, JANA KIRSCHNER  PRVNÍ NOC V NOVÉM BYTĚ              radiozurnal     77
JAROMÍR NOHAVICA                  SAKO ZE SAMETU                      radiozurnal     77
                                  STARÝ MUŽ                           dvojka          76
                                  ZATÍMCO SE KOUPEŠ                   radiozurnal     59
JAROMÍR NOHAVICA/JANA KIRSCHNER   PRVNÍ NOC V NOVÉM BYTĚ              dvojka          47
JAROMÍR NOHAVICA                  TĚŠÍNSKÁ                            dvojka          43
                            

In [31]:
hraje('e pove')

interpret        track                   stanice
RICCHI E POVERI  MADE IN ITALY           dvojka     36
                 COMME VORREI            dvojka     26
                 SARÁ PERCHÉ TI AMO      dvojka     19
                 ACAPULCO                dvojka      6
                 COSA SEI                dvojka      6
                 MAMMA MARIA             dvojka      4
                 SARÁ PERCHE TI AMO      dvojka      4
                 CANZONE D'AMORE         dvojka      2
                 M' INNAMORO DI TE       dvojka      2
                 SARA PERCHE TI AMO      dvojka      2
                 BACIAMOCI               dvojka      1
                 CIAO ITALY, CIAO AMORE  dvojka      1
                 CIAO ITALY,CIAO AMORE   dvojka      1
                 M´INNAMORO DI TE        dvojka      1
                 VOULES VOUZ DANSER      dvojka      1
dtype: int64

In [32]:
hraje('sufjan')

interpret                                                      track                                                      stanice  
SUFJAN STEVENS                                                 VIDEO GAME                                                 radiowave    101
                                                               LONELY MAN OF WINTER (DOVEMAN MIX FT. MELISSA MARY AHERN)  radiowave     95
SUFJAN STEVENS / BRYCE DESSNER / NICO MUHLY / JAMES MCALISTER  SATURN                                                     radiowave     89
                                                               VENUS                                                      radiowave     87
SUFJAN STEVENS                                                 I WALKED                                                   radiowave     82
                                                               WILL ANYBODY EVER LOVE ME                                  radiowave     36
                                  

In [33]:
hraje('grimes')

interpret  track                  stanice  
GRIMES     VIOLENCE FT. I_O       radiowave    82
           FLESH WITHOUT BLOOD    radiowave    81
           4/EM                   radiowave    62
           OBLIVION               radiowave    52
           GO FT. BLOOD DIAMONDS  radiowave    50
           REALITI                radiowave    46
           DELETE FOREVER         radiowave    41
           GENESIS                radiowave    40
           WE APPRECIATE POWER    radiowave    33
dtype: int64

In [34]:
hraje('midi lidi')

interpret  track                    stanice  
MIDI LIDI  LÁSKA                    radiowave    35
           POJĎ VEN (EDIT)          radiowave    35
           LUX                      radiowave    22
                                    vltava        1
           LÁSKA                    dvojka        1
           NA TO NESMÍŠ ZAPOMENOUT  radiowave     1
           PAPÁ                     radiowave     1
           POKOJ                    dvojka        1
dtype: int64

In [35]:
hraje('kazety')

interpret  track  stanice  
KAZETY     JIZVY  radiowave    85
dtype: int64

In [36]:
hraje('margo')

interpret                track                                                                                      stanice  
MARGO                    THE FASCINATING STORY OF TSAR NICHOLAS AND WHAT HAPPENED TO OUR INTERPOLITICAL FRIENDSHIP  radiowave    132
ZELA MARGOSSIAN QUINTET  REFUGE                                                                                     jazz          75
                         ON YA                                                                                      jazz          73
                         TIMELESS                                                                                   jazz          69
MARGO                    A WELL TRAVELED GENTLEMAN MR. PARTESH HAVING A DRINK ON THE 96TH FLOOR, WHAT A VIEW!       radiowave     42
MARGO GURYAN             SUN                                                                                        radiowave      8
dtype: int64

In [37]:
hraje('ventol')

interpret  track          stanice  
VENTOLIN   DISCO SCIENCE  radiowave    96
           PŮJDEME VEN    radiowave    84
           POSUN ČASU     radiowave    61
           SUPERSONIK     radiowave    59
           KAPESNÍK       radiowave     4
           MECH           radiowave     3
           PŮJDEME VEN    dvojka        1
dtype: int64

In [38]:
hraje('post-hudb')

interpret   track                                        stanice  
POST-HUDBA  BEZPEČÍ A KLID                               radiowave    182
            POŘÁD STEJNÝ POUŠTĚ                          radiowave    170
            PŘÍSTAVY                                     radiowave     10
            DO PRÁCE PŘES NUSELÁK, Z PRÁCE PŘES NUSELÁK  vltava         6
            PRVNÍ SNÍH V PODOLÍ                          vltava         5
            BEZPEČÍ A KLID                               vltava         4
            BEZPEČÍ A KLID (VLADISLAV DUBA)              vltava         2
            MODŘÍNOVÝ HÁJ                                vltava         2
dtype: int64

In [39]:
hraje('polachek')

interpret          track                                                 stanice  
CAROLINE POLACHEK  SO COLD YOU'RE HURTING MY FEELINGS                    radiowave    125
                   SMOKE                                                 radiowave     94
                   BUNNY IS A RIDER                                      radiowave     89
                   WELCOME TO MY ISLAND                                  radiowave     89
                   OCEAN OF TEARS                                        radiowave     67
                   DANG                                                  radiowave     30
                   PANG                                                  radiowave      8
                   SO HOT YOURE HURTING MY FEELINGS                      radiowave      4
                   BILLIONS                                              radiowave      1
                   BLOOD AND BUTTER                                      radiowave      1
                 

In [40]:
hraje('gleb')

interpret               track                                   stanice  
GLEB                    ALIBABABIGBASS                          radiowave    121
                        BUDEME TAM FT. VI3E                     radiowave    120
                        GO GO GO                                radiowave     77
                        GAUČ STORYTELLING                       radiowave     74
                        ZEŠLACH CRUNK                           radiowave     70
GLEB ZOO                GAUČ STORYTELLING                       radiowave     64
GLEB                    WORKIN WORKIN                           radiowave     52
                        MC ERIK & BARBARA FT. NATÁLIA HULEJOVÁ  radiowave     37
BIG BOY GLEB            ŠEDÁ HUS                                radiowave      1
GLEB                    BUDEME TAM                              radiowave      1
GLEB X SEPAR            HEY G                                   radiowave      1
GLEB ZOO                GAUČ STORYT

In [41]:
df[(df['interpret'] == 'GLEB ZOO') & (df['stanice'] == 'dvojka')]

,since,id,interpret,interpret_id,track,track_id,stanice,rok
47880,2022-01-06 23:01:12+01:00,12474636,GLEB ZOO,18622,GAUČ STORYTELLING,61548,dvojka,2022


In [42]:
df[(df['interpret'] == 'VENTOLIN') & (df['stanice'] == 'dvojka')]

,since,id,interpret,interpret_id,track,track_id,stanice,rok
7515,2021-01-24 22:33:34+01:00,10474043,VENTOLIN,5344,PŮJDEME VEN,14462,dvojka,2021


In [43]:
df[(df['interpret'] == 'MIDI LIDI') & (df['stanice'] == 'dvojka')]

,since,id,interpret,interpret_id,track,track_id,stanice,rok
7516,2021-01-24 22:47:26+01:00,10474068,MIDI LIDI,317,POKOJ,324,dvojka,2021
18515,2021-04-27 22:46:51+01:00,10883357,MIDI LIDI,317,LÁSKA,1235,dvojka,2021


In [44]:
df[(df['interpret'] == 'RADIOHEAD') & (df['stanice'] == 'dvojka')]

,since,id,interpret,interpret_id,track,track_id,stanice,rok
44432,2021-12-07 22:47:55+01:00,12240162,RADIOHEAD,1267,CREEP,42843,dvojka,2021
77946,2022-09-21 23:48:43+01:00,15522599,RADIOHEAD,1267,CREEP,42843,dvojka,2022


In [45]:
hraje('taylor swift')

interpret                               track                                                stanice    
TAYLOR SWIFT                            CRUEL SUMMER                                         radiowave      136
                                        EXILE FEAT. BON IVER                                 radiowave      103
                                        I CAN SEE YOU                                        radiozurnal     31
                                        ANTI-HERO                                            dvojka          17
                                        CRUEL SUMMER                                         radiozurnal     16
                                        WILLOW                                               dvojka          14
                                        WILDEST DREAMS                                       dvojka           9
                                        LAVENDER HAZE                                        dvojka           8

In [46]:
hraje('něco něco')

interpret  track                                    stanice  
NĚCO NĚCO  ROVINA                                   radiowave    101
           CO VLASTNĚ CHCEŠ                         radiowave     90
           CO VLASTNĚ CHCEŠ II                      radiowave     40
           CHYBY                                    radiowave     12
           MEZERY (TOMÁŠ TKÁČ & ALŽBĚTA TRUSINOVÁ)  vltava         4
dtype: int64

In [47]:
hraje('elefteriadu')

interpret                                track                            stanice
MARTHA ELEFTERIADU                       PROČ PŘICHÁZÍŠ TAK POZDĚ, LÁSKO  pohoda     154
MARTHA A TENA ELEFTERIADU                OD ZÍTŘKA MÁME DOVOLENOU         pohoda     132
                                         A DESKY DÁL STÁRNOU              pohoda     125
                                         TOHLE BOOGIE - WOOGIE            pohoda     115
                                         HREJ DÁL                         pohoda      85
                                         AŤ SE MÚZY POPEROU               pohoda      64
MARTHA A TENA ELEFTERIADU & BOB FRÍDL    ZPÍVEJ TU PÍSEŇ KOUZELNOU        pohoda      36
MARTHA A TENA ELEFTERIADU                MODRÉ KRÁLOVSTVÍ                 pohoda      34
TENA ELEFTERIADU                         PŮLNOČNÍ ČAS                     pohoda      34
MARTHA A TENA ELEFTERIADU & IVAN MLÁDEK  DVĚ PLUS JEDEN                   pohoda      28
MARTHA ELEFTERIADU          

In [48]:
hraje('of montreal')

interpret    track            stanice  
OF MONTREAL  DOUR PERCENTAGE  radiowave    77
             COQUET COQUETTE  radiowave    56
             JIGSAW PUZZLE    radiowave    21
dtype: int64

In [49]:
hraje('mountain goats')

interpret           track                                    stanice  
THE MOUNTAIN GOATS  ANDREW ELDRITCH IS MOVING BACK TO LEEDS  radiowave    170
dtype: int64

In [50]:
hraje('nick cave')

interpret                              track                                                                              stanice    
NICK CAVE, KYLIE MINOGUE               WHERE THE WILD ROSES GROW                                                          radiozurnal    241
NICK CAVE & KYLIE MINOQUE              WHERE THE WILD ROSES GROW                                                          dvojka          50
NICK CAVE AND THE BAD SEEDS            WE NO WHO U R (NICK CAVE & WARREN ELLIS)                                           vltava          11
KYLIE MINOGUE, NICK CAVE               WHERE THE WILD ROSES GROW                                                          radiozurnal      8
NICK CAVE                              IDIOT PRAYER                                                                       vltava           7
                                       PALACES OF MONTEZUMA (NICK CAVE & WARREN ELLIS & MARTYN P. CASEY & JIM SCLAVUNOS)  vltava           7
                    

## Hledání písní

In [51]:
df[df['track'].str.lower().str.contains('vražd',na=False)][['interpret','track']].drop_duplicates()

,interpret,track
158004,NOČNÍ OPTIKA,VRAŽDA V ORIENT EXPRESSU


## Žebříček pro článek

In [52]:
df['den'] = df['since'].dt.dayofyear

In [53]:
df['den'].nunique()

366

In [74]:
top10rz = pd.DataFrame(df[(df['stanice'] == 'radiozurnal') & (df['rok'] == 2023)].groupby('interpret')['id'].nunique().nlargest(6))

In [75]:
df.columns

Index(['since', 'id', 'interpret', 'interpret_id', 'track', 'track_id',
       'stanice', 'rok', 'den'],
      dtype='object')

In [76]:
top10rz

,id
interpret,
ROBBIE WILLIAMS,694
CHINASKI,627
ED SHEERAN,627
R.E.M.,592
U 2,590
COLDPLAY,555


In [77]:
top10rz = top10rz.reset_index()

In [78]:
top10rz

,interpret,id
0,ROBBIE WILLIAMS,694
1,CHINASKI,627
2,ED SHEERAN,627
3,R.E.M.,592
4,U 2,590
5,COLDPLAY,555


In [79]:
dni2023 = df[df['rok'] == 2023]['den'].nunique()

In [80]:
top10rz['denne'] = top10rz['id'] / dni2023

In [81]:
top10rz['denne'] = top10rz['denne'].apply(lambda x: round(x, 2))

In [82]:
top10rz['Písní'] = top10rz['interpret'].apply(lambda x: df[(df['stanice'] == 'radiozurnal') & (df['rok'] == 2023) & (df['interpret'] == x)]['track'].nunique())

In [83]:
top10rz

,interpret,id,denne,Písní
0,ROBBIE WILLIAMS,694,2.22,12
1,CHINASKI,627,2.00,20
2,ED SHEERAN,627,2.00,11
3,R.E.M.,592,1.89,11
4,U 2,590,1.88,10
5,COLDPLAY,555,1.77,10


In [84]:
hezke_nazvy = {"ROBBIE WILLIAMS": "Robbie Williams", "CHINASKI": "Chinaski", "ED SHEERAN": "Ed Sheeran", "U 2": "U2", "COLDPLAY": "Coldplay", "P!NK": "P!nk", "ONE REPUBLIC": "One Republic", "ANETA LANGEROVÁ": "Aneta Langerová", "ADELE": "Adele", "ROXETTE": "Roxette", "CRANBERRIES": "Cranberries"}

In [85]:
top10rz['interpret'] = top10rz['interpret'].replace(hezke_nazvy)

In [86]:
top10rz

,interpret,id,denne,Písní
0,Robbie Williams,694,2.22,12
1,Chinaski,627,2.00,20
2,Ed Sheeran,627,2.00,11
3,R.E.M.,592,1.89,11
4,U2,590,1.88,10
5,Coldplay,555,1.77,10


In [87]:
top10rz = top10rz.rename(columns={'interpret': 'Interpret', 'id': 'Přehrání', 'denne': 'Přehrání za den'})

In [88]:
top10rz = top10rz.drop(columns=['Přehrání'])

In [89]:
top10rz

,Interpret,Přehrání za den,Písní
0,Robbie Williams,2.22,12
1,Chinaski,2.00,20
2,Ed Sheeran,2.00,11
3,R.E.M.,1.89,11
4,U2,1.88,10
5,Coldplay,1.77,10


In [90]:
wikid = pd.read_json(os.path.join("wikidata","wikidata.json"))
wikid['interpret_hlavni'] = wikid['interpret_hlavni'].apply(lambda x: x.lower())

In [91]:
wikid.columns

Index(['interpret_hlavni', 'id', 'stazeno', 'nazev_wiki', 'popis_en',
       'popis_cs', 'velikost', 'zanry', 'datum_narozeni', 'datum_umrti',
       'zacatek_pusobeni', 'konec_pusobeni', 'zalozeni', 'zanik', 'gender',
       'misto_puvodu', 'zeme_puvodu', 'obcanstvi', 'etnicita', 'hlas',
       'nastroje', 'oceneni', 'nabozenstvi', 'orientace', 'spotify_id',
       'spotify_odberatele'],
      dtype='object')

In [92]:
def zacatek(i):
        if i == 'One Republic':
            return 2002
        else:
            return int(wikid[wikid['interpret_hlavni'] == i.lower()]['zacatek_pusobeni'].min())

In [93]:
top10rz['Začátek kariéry'] = top10rz['Interpret'].apply(lambda x: zacatek(x))

In [95]:
top10rz['Přehrání za den'] = top10rz['Přehrání za den'].apply(lambda x: str(x).replace(".",","))

In [97]:
top10rz['Přehrání za den'] = top10rz['Přehrání za den'].apply(lambda x: str(x).replace(",0",",00"))

In [98]:
top10rz

,Interpret,Přehrání za den,Písní,Začátek kariéry
0,Robbie Williams,"2,22",12,1990
1,Chinaski,"2,00",20,1987
2,Ed Sheeran,"2,00",11,2004
3,R.E.M.,"1,89",11,1980
4,U2,"1,88",10,1976
5,Coldplay,"1,77",10,1997


In [99]:
tabulka(top10rz, titulek="Nejhranější interpreti na stanici ČRo Radiožurnál v roce 2023", poradi=True)

'<table class="dataframe table table--responsive table--w100p table--striped-red table--plain"><caption>Nejhranější interpreti na stanici ČRo Radiožurnál v roce 2023</caption><thead><tr style="text-align: right;"><th class="text-nowrap"></th><th class="text-nowrap">Interpret</th><th class="text-nowrap">Přehrání za den</th><th class="text-nowrap">Písní</th><th class="text-nowrap">Začátek kariéry</th></tr></thead><tbody class="text-sm"><tr><td>1.</td><td>Robbie Williams</td><td>2,22</td><td>12</td><td>1990</td></tr><tr><td>2.</td><td>Chinaski</td><td>2,00</td><td>20</td><td>1987</td></tr><tr><td>3.</td><td>Ed Sheeran</td><td>2,00</td><td>11</td><td>2004</td></tr><tr><td>4.</td><td>R.E.M.</td><td>1,89</td><td>11</td><td>1980</td></tr><tr><td>5.</td><td>U2</td><td>1,88</td><td>10</td><td>1976</td></tr><tr><td>6.</td><td>Coldplay</td><td>1,77</td><td>10</td><td>1997</td></tr></tbody></table>'